## 네이버 스포츠 뉴스 웹크롤링 , 워드 클라우드 시각화

In [37]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
# from urllib.parse import quote
# from tqdm import tqdm
# import re, nltk
# from konlpy.tag import Okt 
# import numpy as np
# from PIL import Image
# import matplotlib.pyplot as plt
# from wordcloud import WordCloud
# plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['axes.unicode_minus'] = False

- 네이버 스포츠 뉴스 크롤링

In [38]:
sports = ['kbaseball', 'wbaseball','kfootball','wfootball', 'basketball','volleyball','golf','general']
base_url = 'https://sports.news.naver.com/'
suffix_url= '/news/index?isphoto=N&page='

In [39]:
driver = webdriver.Chrome()
url = f'{base_url}kbaseball{suffix_url}1'
driver.get(url)

In [40]:
# 기사 목록
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.news_list > ul > li')
len(lis)

20

In [41]:
# 기사 제목
li = lis[0]
title = li.select_one('.title').get_text().strip()
title

'AG 출전 의지 강했던 132억 좌완 에이스, 결국 출전 불발…김영규 대체 발탁, 이정후→김성윤 교체 [MK항저우]'

- 종목별로 몇페이지가 있는지 확인하기

In [42]:
# 10페이지가 넘는 종목 선택
url = f'{base_url}wfootball{suffix_url}1'
driver.get(url)

In [43]:
# 모든 페이지마다 끝나는 조건을 다음을 눌르고 다음 페이지가 없을 때 까지.
sports_dict ={}
for sport in sports:
    url = f'{base_url}{sport}{suffix_url}1'
    driver.get(url)
    time.sleep(2) 

    next_clicked=0
    while True:
        try:
            driver.find_element(By.CSS_SELECTOR, '.paginate > .next').click()
            next_clicked += 1
            time.sleep(2)
        except:     # 모든 종목의 마지막 페이지가 있는 화면
            pages = driver.find_elements(By.CSS_SELECTOR, '.paginate > a')
            if next_clicked == 0:
                num_pages = len(pages) + 1
            else:
                num_pages = next_clicked * 10 + len(pages)
            sports_dict[sport] = num_pages
            break
sports_dict

{'kbaseball': 7,
 'wbaseball': 5,
 'kfootball': 7,
 'wfootball': 14,
 'basketball': 5,
 'volleyball': 5,
 'golf': 3,
 'general': 10}

- 모든 종목, 모든 페이지에서 대해서 타이틀 가져오기

In [51]:
sports_dict ={}
    # 제목만 따오기 
for sport, num_pages in sports_dict.items():
    title = li.select_one('.title').get_text().strip()
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.news_list > ul > li')
    for sport in sports:
        url = f'{base_url}{sport}{suffix_url}1'
        driver.get(url)
        time.sleep(2) 

        next_clicked=0
        while True:
            try:
                driver.find_element(By.CSS_SELECTOR, '.paginate > .next').click()
                next_clicked += 1
                time.sleep(2)
            except:     # 모든 종목의 마지막 페이지가 있는 화면
                pages = driver.find_elements(By.CSS_SELECTOR, '.paginate > a')
                if next_clicked == 0:
                    num_pages = len(pages) + 1
                else:
                    num_pages = next_clicked * 10 + len(pages)
                sports_dict[sport] = num_pages
                break
        
sports_dict

{}